Statistical errors of CNN models

#read current
#load model and calculate out put od model
#plot 3d error of Bfield

In [1]:
import numpy as np
import torch
if torch.cuda.device_count():
    device = 'cuda'
    print('Good to go')
else:
    device = 'cpu'
    print('Using cpu')
    
from Neural_network import Generative_net, ResidualEMNSBlock_3d, BigBlock, weight_init, eMNS_Dataset
from Training_loop import train_part_GM,get_mean_of_dataloader


Good to go


In [35]:
num_input = 12
output_shape = (3,20,20,20)
SB_args = (64,64,4) # (Cin, Cout, num_block)
BB_args = (2,2) # (scale_factor, num_block)
SB_block = ResidualEMNSBlock_3d 
BB_block = BigBlock

Generative_network = Generative_net(SB_args, BB_args, SB_block, BB_block, num_input=num_input, output_shape= output_shape)
Generative_network = torch.load('EMS_CNN.pt', map_location="cuda:0")
Generative_network.eval()
Generative_network.to(device)





Generative_net(
  (proj): Linear(in_features=12, out_features=8000, bias=True)
  (conv3d): Conv3d(64, 3, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (total_net): Sequential(
    (0): Linear(in_features=12, out_features=8000, bias=True)
    (1): Unflatten(dim=1, unflattened_size=(64, 5, 5, 5))
    (2): BigBlock(
      (block): Sequential(
        (0): ResidualEMNSBlock_3d(
          (block): Sequential(
            (0): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
            (2): LeakyReLU(negative_slope=0.01)
          )
          (shortcut): Identity()
        )
        (1): ResidualEMNSBlock_3d(
          (block): Sequential(
            (0): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
            (2): LeakyReLU(ne

In [37]:
from ReadData import ReadCurrentAndField_CNN
foldername="./Data/"
filepattern = "MagneticField[0-9]*.txt"
current,data = ReadCurrentAndField_CNN (foldername,filepattern)
position = data[:,0:3,0:20,0:20,0:20]
Bfield = data[:,3:,0:20,0:20,0:20]

In [44]:
print(current.shape)

test_current=current[0,:]

current.to(device)

print(current.device) 
input=current.cuda(0)
print(input.device) 
a=input[0,:]

print(a.shape[1])
# Generative_network(b)





torch.Size([1000, 12])
cpu
cuda:0


IndexError: tuple index out of range